# Restrict access to Power BI model data
As a data modeler, you set up RLS by creating one or more roles. A role has a unique name in the model, and it usually includes one or more rules. Rules enforce filters on model tables by using Data Analysis Expressions (DAX) filter expressions.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> By default, a data model has no roles. A data model without roles means that users (who have permission to query the data model) have access to all model data.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> It's possible to define a role that includes no rules. In this case, the role provides access to all rows of all model tables. This role set up would be suitable for an admin user who is allowed to view all data.

You can create, validate, and manage roles in Power BI Desktop. For Azure Analysis Services or SQL Server Analysis Services models, you can create, validate, and manage roles by using SQL Server Data Tools (SSDT).

You can also create and manage roles by using SQL Server Management Studio (SSMS), or by using a third-party tool, like [Tabular Editor](https://tabulareditor.com/).

To gain a better understanding of how RLS restricts access to data, watch the following animated image.

<img src="../images/05_Work with semantic models in Microsoft Fabric/04/introduce-row-level-security.gif" alt="Animated diagram demonstrates how row-level security works for two users who each have access to specific country data." style="border: 2px solid black; border-radius: 10px;">

## Apply star schema design principles
We recommend you apply [star schema](https://learn.microsoft.com/en-us/power-bi/guidance/star-schema) design principles to produce a model comprising dimension and fact tables. It’s common to set up Power BI to enforce rules that filter dimension tables, allowing [model relationships](https://learn.microsoft.com/en-us/power-bi/transform-model/desktop-relationships-understand) to efficiently propagate those filters to fact tables.

The following image is the model diagram of the Adventure Works sales analysis data model. It shows a star schema design comprising a single fact table named **Sales**. The other tables are dimension tables that support the analysis of sales measures by date, sales territory, customer, reseller, order, product, and salesperson. Notice the model relationships connecting all tables. These relationships propagate filters (directly or indirectly) to the **Sales** table.

<img src="../images/05_Work with semantic models in Microsoft Fabric/04/model-diagram-star-schema.png" alt="Screenshot shows a Power BI Desktop model diagram comprising the tables and relationships as described in the previous paragraph." style="border: 2px solid black; border-radius: 10px;">

This model design supports examples presented in this unit.

## Define rules
Rule expressions are evaluated within row context. Row context means the expression is evaluated for each row using the column values of that row. When the expression returns TRUE, the user can “see” the row.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> To learn more about row context, work through the [Add calculated tables and columns to Power BI Desktop models module](https://learn.microsoft.com/en-us/training/modules/dax-power-bi-add-calculated-tables/). While this module describes adding model calculations, it includes a unit that introduces and describes row context.

You can define rules that are either static or dynamic.

### Static rules
Static rules use DAX expressions that refer to constants.

Consider the following rule applied to the Region table that restricts data access to Midwest sales.

In [0]:
#DAX
'Region'[Region] = "Midwest"

The following steps explain how Power BI enforces the rule. It:
1. Filters the **Region** table, resulting in one visible row (for Midwest).
2. Uses the model relationship to propagate the **Region** table filter to the **State** table, resulting in 14 visible rows (the Midwest region comprise 14 states).
3. Uses the model relationship to propagate the **State** table filter to the Sales table, resulting in thousands of visible rows (the sales facts for the states that belong to the Midwest region).

The simplest static rule that you can create restricts access to all table rows. Consider the following rule applied to the **Sales Details** table (not depicted in the model diagram).

In [0]:
#DAX
FALSE()

This rule ensures users can't access any table data. It could be useful when salespeople are allowed to see aggregated sales results (from the **Sales** table) but not order-level details.

Defining static rules is simple and effective. Consider using them when you need to create only a few of them, as might be the case at Adventure Works where there are only six US regions. However, be aware of disadvantages: setting up static rules can involve significant effort to create and set up. It would also require you to update and republish the dataset when new regions are onboarded.

If there are many rules to set up and you anticipate adding new rules in the future, consider creating dynamic rules instead.

### Dynamic rules
Dynamic rules use specific DAX functions that return environmental values (as opposed to constants). Environmental values are returned from three specific DAX functions:
- [USERNAME](https://learn.microsoft.com/en-us/dax/username-function-dax) or [USERPRINCIPALNAME](https://learn.microsoft.com/en-us/dax/userprincipalname-function-dax) – Returns the Power BI authenticated user as a text value.
- [CUSTOMDATA](https://learn.microsoft.com/en-us/dax/customdata-function-dax) - Returns the **CustomData** property passed in the connection string. Non-Power BI reporting tools that connect to the dataset by using a connection string can set this property, like Microsoft Excel.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> Be aware that the USERNAME function returns the user in the format of DOMAIN\username when used in Power BI Desktop. However, when used in the Power BI service, it returns the format of the user's User Principal Name (UPN), like username@adventureworks.com. Alternatively, you can use the USERPRINCIPALNAME function, which always returns the user in the user principal name format.

Consider a revised model design that now includes the (hidden) **AppUser** table. Each row of the **AppUser** table describes a username and region. A model relationship to the **Region** table propagates filters from the **AppUser** table.

<img src="../images/05_Work with semantic models in Microsoft Fabric/04/model-diagram-appuser-table.png" alt="Image shows a revised model diagram that now includes the AppUser table. This table has two columns: Region and User Name." style="border: 2px solid black; border-radius: 10px;">

The following rule applied to the AppUser table restricts data access to the region(s) of the authenticated user.

In [0]:
#DAX
'AppUser'[UserName] = USERPRINCIPALNAME()

Defining dynamic rules is simple and effective when a model table stores username values. They allow you to enforce a data-driven RLS design. For example, when salespeople are added to, or removed from, the AppUser table (or are assigned to different regions), this design approach just works.

## Validate roles
When you create roles, it’s important to test them to ensure they apply the correct filters. For data models created in Power BI Desktop, there’s the **View as** function that allows you to see the report when different roles are enforced, and different username values are passed.

<img src="../images/05_Work with semantic models in Microsoft Fabric/04/power-bi-desktop-security-view.png" alt="Screenshot shows the Power B I Desktop Modeling ribbon. The “View as” command is highlighted." style="border: 2px solid black; border-radius: 10px;">

## Set up role mappings
Role mappings must be set up in advance of users accessing Power BI content. Role mapping involves assigning Microsoft Entra security objects to roles. Security objects can be user accounts or security groups.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> When possible, it’s a good practice to map roles to security groups. That way, there will be fewer mappings, and you can delegate the group membership management to the network administrators.

For Power BI Desktop developed models, role mapping is typically done in the Power BI service. For Azure Analysis Services or SQL Server Analysis Services models, role mapping is typically done in SSMS.

For more information about setting up RLS, see:
- [Row-level security (RLS) with Power BI](https://learn.microsoft.com/en-us/power-bi/enterprise/service-admin-rls)
- [Row-level security (RLS) guidance in Power BI Desktop](https://learn.microsoft.com/en-us/power-bi/guidance/rls-guidance)

## Use single sign-on (SSO) for DirectQuery sources
When your data model has DirectQuery tables and their data source supports SSO, the data source can enforce data permissions. This way, the database enforces RLS, and Power BI datasets and reports honor the data source security.

Consider that Adventure Works has an Azure SQL Database for their sales operations that resides in the same tenant as Power BI. The database enforces RLS to control access to rows in various database tables. You can create a DirectQuery model that connects to this database without roles and publish it to the Power BI service. When you set the data source credentials in the Power BI service, you [enable SSO](https://learn.microsoft.com/en-us/power-bi/connect-data/service-azure-sql-database-with-direct-connect). When report consumers open Power BI reports, Power BI passes their identity to the data source. The data source then enforces RLS based on the identity of the report consumer.

<img src="../images/05_Work with semantic models in Microsoft Fabric/04/set-data-source-credentials-single-sign-on.png" alt="Screenshot shows the data source credentials window with the S O option enabled." style="border: 2px solid black; border-radius: 10px;">

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> For information about Azure SQL Database RLS, see [Row-level security](https://learn.microsoft.com/en-us/sql/relational-databases/security/row-level-security).

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> Calculated tables and calculated columns that reference a DirectQuery table from a data source with SSO authentication aren’t supported in the Power BI service.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> For more information about data sources that support SSO, see [Single sign-on (SSO) for DirectQuery sources](https://learn.microsoft.com/en-us/power-bi/connect-data/power-bi-data-sources).

# Restrict access to Power BI model objects
As a data modeler, you can consider restricting user access to Power BI model objects. Object-level security (OLS) can restrict access to specific tables and columns, and their metadata. Typically, you apply OLS to secure objects that store sensitive data, like employee personal data.

When Power BI enforces OLS, not only does it restrict access to tables and columns, but it can also secure metadata. When you secure metadata, it’s not possible to retrieve information about secured tables and columns by using [Dynamic Management Views (DMVs)](https://learn.microsoft.com/en-us/analysis-services/instances/use-dynamic-management-views-dmvs-to-monitor-analysis-services).

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> Tabular models can hide tables and columns (and other objects) by using a [perspective](https://learn.microsoft.com/en-us/analysis-services/tabular-models/perspectives-ssas-tabular). A perspective defines viewable subsets of model objects to help provide a specific focus for report authors. Perspectives are intended to reduce the complexity of a model, helping report authors find resources of interest. However, perspectives aren’t a security feature because they don’t secure objects. A user can still query a table or column even when it’s not visible to them.

Consider an example at Adventure Works. This organization has a data warehouse dimension table named **DimEmployee**. The table includes columns that store employee name, phone, email address, and salary. While general report consumers can see employee name and contact details, they must not be able to see salary values. Only senior Human Resources staff are permitted to see salary values. So, the data modeler used OLS to grant access to the salary column only to specific Human Resources staff.

<img src="../images/05_Work with semantic models in Microsoft Fabric/04/employee-table-restrict-salary-column-object-level-security.png" alt="Screenshot shows a model diagram view of the Employee table, which includes the restricted Salary column." style="border: 2px solid black; border-radius: 10px;">

OLS is a feature inherited from Azure Analysis Services (AAS) and SQL Server Analysis Services (SSAS). The feature is available in Power BI Premium to provide backward compatibility for models migrated to Power BI. For this reason, it’s not possible to completely set up OLS in Power BI Desktop.

## Set up OLS
To set up OLS, you start by creating roles. You can create roles in Power BI Desktop in the same way you do when setting up RLS. Next, you need to add OLS rules to the roles. This capability isn’t supported by Power BI Desktop, so you’ll need to take a different approach.

You add OLS rules to a Power BI Desktop model by using an [XML for Analysis (XMLA) endpoint](https://learn.microsoft.com/en-us/power-bi/enterprise/service-premium-connect-tools). XMLA endpoints are available with Power BI Premium, and they provide access to the Analysis Services engine in the Power BI service. The read/write endpoint supports dataset management, application lifecycle management, advanced data modeling, and more. You can use XMLA endpoint-enabled APIs for scripting, such as [Tabular Model Scripting Language (TMSL)](https://learn.microsoft.com/en-us/analysis-services/tmsl/tabular-model-scripting-language-tmsl-reference) or the [PowerShell SqlServer module](https://www.powershellgallery.com/packages/SqlServer/). Or you can use a client tool, like SSMS. There are third-party tool options too, like [Tabular Editor](https://tabulareditor.github.io/TabularEditor/), which is an open-source tool for creating, maintaining, and managing models.

By default, all model tables and columns aren’t restricted. You can set them to **None** or **Read**. When set to **None**, users associated with the role can’t access the object. When set to **Read**, users associated with the role can access the object. When you’re restricting specific columns, ensure the table isn’t set to **None**.

Once you’ve added the OLS rules, you can publish the model to the Power BI service. Use the same process for RLS to map accounts and security groups to the roles.

## Considerations
In a Power BI report, when a user doesn’t have permission to access a table or column, they'll receive an error message. The message will inform them that the object doesn’t exist.

<img src="../images/05_Work with semantic models in Microsoft Fabric/04/fields-need-to-fixed-object-level-security.png" alt="Screenshot shows a Power B I Desktop error message when a report visual attempts to query a restricted column." style="border: 2px solid black; border-radius: 10px;">

Consider carefully whether OLS is the right solution for your project. When a user opens a Power BI report that queries a restricted object (for them), the error message could be confusing and will result in a negative experience. To them, it looks like the report is broken. A better approach might be to create a separate set of models or reports for the different report consumer requirements.

## Restrictions
There are restrictions to be aware of when implementing OLS.

You can’t mix RLS and OLS in the same role. If you need to apply RLS and OLS in the same model, create separate roles dedicated to each type. Also, you can’t set table-level security if it breaks a relationship chain. For example, if there are relationships between tables A and B, and B and C, you can't secure table B. If table B is secured, a query on table A can't transit the relationships between table A and B, and B and C. In this case, you could set up a separate relationship between tables A and C.

<img src="../images/05_Work with semantic models in Microsoft Fabric/04/model-relationships-object-level-security.png" alt="Diagram shows the relationship example described in the previous paragraph." style="border: 2px solid black; border-radius: 10px;">

However, model relationships that reference a secured column will work, providing that the column’s table isn’t secured.

Lastly, while it isn’t possible to secure measures, a measure that references secured objects is automatically restricted.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> For more information, see [Object-level security](https://learn.microsoft.com/en-us/analysis-services/tabular-models/object-level-security).

# Apply good modeling practices
It’s critical that your model enforces data permissions correctly and efficiently. The following list provides you with good development practices to apply.
- Strive to define fewer datasets (data models) with well-designed roles.
- Strive to create fewer roles by using dynamic rules. A data-driven solution is easier to maintain because you don’t need to add new roles.
- When possible, create rules that filter dimension tables instead of fact tables. It will help to deliver faster query performance.
- Validate that the model design, including its relationships and relationship properties, are correctly set up.
- Use the `USERPRINCIPALNAME` function instead of `USERNAME` function. It provides consistency when validating the roles in Power BI Desktop and the Power BI service.
- Rigorously validate RLS and OLS by testing all roles.
- Ensure that the Power BI Desktop data source connection uses the same credentials that will be applied when set up in the Power BI service.